### GNN

In [9]:
import torch.nn as nn
import torch.nn.functional as F
import torch

import torchvision.transforms as transforms
from torchvision import datasets

import numpy as np
import os

In [26]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        
        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers
        
        self.model = nn.Sequential(
            *block(100, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, 28*28),
            nn.Tanh()
        )
        
    def forward(self, x):
        img = self.model(x)
        img = img.view(img.size(0), 1, 28, 28)
        return img
    
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )
        
    def forward(self, img):
        x = self.model(img.view(img.size(0),-1))
        return x
 

generator = Generator()
discriminator = Discriminator()

In [27]:
loss_func = nn.BCELoss()
generator.cuda()
discriminator.cuda()
loss_func.cuda()


BCELoss()

In [35]:
optimizer_G = torch.optim.Adam(generator.parameters(), lr=1e-3, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=1e-3, betas=(0.5, 0.999))

from torch.autograd import Variable

In [13]:
dl = torch.utils.data.DataLoader(
    datasets.MNIST(
        './data', download=False,
        transform=transforms.Compose(
            [transforms.Resize((28, 28)),
             transforms.ToTensor(),
             transforms.Normalize([0.5], [0.5])
                                
            ]
        ),
    ),
    batch_size=128,
    shuffle=True
)

In [45]:
epoches = 20
generator.parameters()
if not os.path.exists('./images'):
    os.mkdir('./images')
from torchvision.utils import save_image
import time

for epoch in range(epoches):
    for i, (imgs, _) in enumerate(dl):
        valid = Variable(torch.cuda.FloatTensor(imgs.size(0),1).fill_(1.0), requires_grad=False)
        fake = Variable(torch.cuda.FloatTensor(imgs.size(0),1).fill_(0.0), requires_grad=False)
        
        real_imgs = Variable(imgs.type(torch.cuda.FloatTensor))  # ??
        optimizer_G.zero_grad()
        
        z = Variable(torch.cuda.FloatTensor(np.random.normal(0, 1, (imgs.shape[0], 100) ) ) )
        
        gen_imgs = generator(z)
        
        g_loss = loss_func(discriminator(gen_imgs) , valid)
        
        g_loss.backward()
        optimizer_G.step()
        
        ##
        optimizer_D.zero_grad()
        r_loss = loss_func(discriminator(real_imgs), valid)
        f_loss = loss_func(discriminator(gen_imgs.detach()), fake)
        
        d_loss = (r_loss + f_loss) /2 ## using torch add_ ??
        d_loss.backward()
        optimizer_D.step()
        
        # print
        if i % 100 == 0:
            print("epoch %d/%d, batch %d/%d, [D loss = %f] [G loss = %f] " %(
                (epoch, epoches, i, len(dl), d_loss.item(), g_loss.item())
            ))
            #
            save_image(gen_imgs[:25], "./images/image_%01d_%04d_%s.png"%(epoch, i, time.strftime('%m-%d_%H.%M', time.localtime())), nrow=5, normalize=True)
        

epoch 0/20, batch 0/469, [D loss = 0.566685] [G loss = 1.257586] 
epoch 0/20, batch 100/469, [D loss = 0.683367] [G loss = 0.528397] 
epoch 0/20, batch 200/469, [D loss = 0.548980] [G loss = 1.035666] 
epoch 0/20, batch 300/469, [D loss = 0.506490] [G loss = 0.904833] 
epoch 0/20, batch 400/469, [D loss = 0.577181] [G loss = 1.186145] 
epoch 1/20, batch 0/469, [D loss = 0.501223] [G loss = 1.119981] 
epoch 1/20, batch 100/469, [D loss = 0.578340] [G loss = 1.339372] 
epoch 1/20, batch 200/469, [D loss = 0.528464] [G loss = 0.939053] 
epoch 1/20, batch 300/469, [D loss = 0.570555] [G loss = 0.753809] 
epoch 1/20, batch 400/469, [D loss = 0.599229] [G loss = 0.844490] 
epoch 2/20, batch 0/469, [D loss = 0.592245] [G loss = 1.086094] 
epoch 2/20, batch 100/469, [D loss = 0.578085] [G loss = 1.150848] 
epoch 2/20, batch 200/469, [D loss = 0.543119] [G loss = 1.184411] 
epoch 2/20, batch 300/469, [D loss = 0.595494] [G loss = 0.692129] 
epoch 2/20, batch 400/469, [D loss = 0.587552] [G loss

In [43]:
time.strftime('%m-%d_%H.%M', time.localtime())

NameError: name 'time' is not defined

In [39]:
torch.cuda.empty_cache()